In [95]:
import tweepy
import random
#from our keys module (keys.py), import the keys dictionary

CONSUMER_KEY = "HGsCa24B9jlGFiDpRaBemTyjj"
CONSUMER_SECRET = "BA87yf8QyolGnTUmHEc07A1ZfnKbmHa6GfMRj3tBEJ8lWMFGPH"
ACCESS_TOKEN = "1006801061754097664-I7ziYNHGJ3ydUPgtEP0wYEvTBYuWJz"
ACCESS_TOKEN_SECRET = "Xz3kwLMXd7UYDpkyjNMfXn5ATnW3njLNVuybST3ci5v3k"

#CONSUMER_KEY = "tNq5rXzQtpCfQN5SqhvYDjx84"
#CONSUMER_SECRET = "3KTXCM4jVgizY1MCPHCm02oxV21OBMdX2jGAfY7mZ27aSbwveF"
#ACCESS_TOKEN = "1003684959419322371-2yvCMWHGhs7S32TIgEY9wV9SziKg11"
#ACCESS_TOKEN_SECRET = "MyIrkq24K7R7WehRLeRkpVCePbmQB7L56He65gfeaLgAw"


auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [13]:
import itunes
import unirest
import tweet_dumper
import time

In [14]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer, MaxAbsScaler
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [15]:
df= pd.read_csv("m.csv")

## Create Non-negative Matrix reccomender

### This method decomposes the corpora into linearly independent topic

In [61]:
pod_descriptions = df.iloc[:,11]
pod_descriptions= pod_descriptions.dropna()
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(pod_descriptions)
# Create an NMF instance: model
n_components = 2
model = NMF(n_components=n_components)

# Fit the model to articles
model.fit(count_train)

# Transform the articles: nmf_features
nmf_features =model.transform(count_train)

# Print the NMF features
print(nmf_features.shape)

print("####Success Creating the NMF with n_components = " + str(n_components))

(6747, 2)
####Success Creating the NMF with n_components = 2


## Get A wild card  podcast for new users in Twitter

### This kernel should be amplified to most popular

In [74]:
podcast = itunes.lookup(160904630)
podcast_url = podcast.get_url()

In [75]:
print(podcast_url)

https://itunes.apple.com/us/podcast/ted-talks-daily/id160904630?mt=2&uo=4


## Helper Function to get Twitter History of User

### Todo : maybe limit total number of tweets?

In [48]:
screen_name = 'BarackObama'
new_tweets = api.user_timeline(screen_name = screen_name,count=200)
user = api.get_user(screen_name)

friends = user.friends()
#print(user.screen_name)
all_friends = []
for friend in friends:
    all_friends.append(friend.screen_name)

In [108]:
def get_user_tweets(screen_name):
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    try:
        user = api.get_user(screen_name)
    except tweepy.error.TweepError as e:
        time.sleep(180)
        user = api.get_user(screen_name)
    try:
        friends = user.friends()
        print(user.followers_count)
    except tweepy.error.TweepError as e:
        time.sleep(180)
        friends = user.friends()
    #print(user.screen_name)
    all_friends = []
    for friend in friends:
        all_friends.append(friend.screen_name)
    
    alltweets = []
    if len(new_tweets) < 1:
        
        return []
    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one

    if len(new_tweets) > 0:
        
        oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        #print "getting tweets before %s" % (oldest)

    #all subsiquent requests use the max_id param to prevent duplicates
        try:
            new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
                
        except tweepy.error.TweepError as e:
            time.sleep(180)
            new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
            
            
    #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        #print(oldest)

        #print "...%s tweets downloaded so far" % (len(alltweets))

        #transform the tweepy tweets into a 2D array that will populate the csv 
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
    return all_friends,outtweets

In [109]:
h_tags,my_tweets = get_user_tweets('BarackObama')

103235794


In [98]:
#for i in range(1):
 #   if i%5 ==0:
  #      print i 
a,b = get_user_tweets("jandresmillan")

print(a)

[u'Noahsyndergaard', u'spurs', u'kobebryant', u'TheAdamsEra', u'StephenAtHome']


In [111]:
def get_from_hashtag(all_hashtags):
    all_bi_tokens = []
    for h_tag in all_hashtags:
        
        trial_bitoken = split_hashtag(h_tag)
       # print(trial_bitoken)
        if len(trial_bitoken) > 1:
            all_bi_tokens.append(trial_bitoken)
    
    hash_signal = random.choice(all_bi_tokens)
    final_hash_signal = hash_signal[0]
    h_tag_user = "@" + hash_signal[0]
    
    
    
    Celebrity = True;
    while Celebrity:
        for i in range(1,len(hash_signal)):
            final_hash_signal = final_hash_signal + "+" + hash_signal[i]
            h_tag_user = h_tag_user + hash_signal[i]
        print(h_tag_user)
        try:
            user = api.get_user(h_tag_user)
        
        except tweepy.error.TweepError as e:
            time.sleep(180)
            user = api.get_user(h_tag_user)
        try:
            
            nbr_friends = user.followers_count
            print(nbr_friends)
            
        except tweepy.error.TweepError as e:
            time.sleep(180)
            nbr_friends = user.followers_count
            
        if nbr_friends > 1000000:
            Celebrity = False;
            
        else:
            hash_signal = random.choice(all_bi_tokens)
            final_hash_signal = hash_signal[0]
            h_tag_user = "@" + hash_signal[0]
        
        
    return h_tag_user,final_hash_signal
a1,a2 = get_from_hashtag(a)
print(a1,a2)

@StephenAtHome
18505110
(u'@StephenAtHome', u'Stephen+At+Home')


In [72]:
obamatweets = pd.read_csv('BarackObama_tweets.csv')

In [73]:
obamatweets.text

0       RT @MichelleObama: Sometimes truth transcends ...
1       Congratulations to the @Capitals! This @NHLBla...
2       “Low plastic stool, cheap but delicious noodle...
3       This National Gun Violence Awareness Day, show...
4       We can never truly repay the debt we owe our f...
5       This Center is for the leaders of tomorrow who...
6       Happy Mother’s Day to every mom out there, esp...
7       There are few issues more important to the sec...
8       RT @MichelleObama: Happy #CollegeSigningDay! S...
9       These talented young people will create a bett...
10      Our statement on the passing of Former First L...
11      RT @ObamaFoundation: We just announced the ina...
12      RT @ObamaFoundation: This week—50 years since ...
13      Incredible to have a Chicago team in the Final...
14      Michelle and I are so inspired by all the youn...
15      Our most important task as a nation is to make...
16      In Singapore with young people who are advocat...
17      41: I 

In [38]:
import tweethon

In [39]:
api = tweethon.Api() 


HTTPError: HTTP Error 404: Not Found

In [88]:
import re
def split_hashtag(hashtagestring):
    fo = re.compile(r'#[A-Z]{2,}(?![a-z])|[A-Z][a-z]+')
    fi = fo.findall(hashtagestring)
    result = ''
    for var in fi:
        result += var + ' '
       # print (result)
    result = result.split()
    return result;

In [30]:
split_hashtag(a[4])

Stephen 
Stephen At 
Stephen At Home 


[u'Stephen', u'At', u'Home']

## PodFather TwitBot

### Main Body

In [129]:
#Get history of tweets 
#public_tweets = api.home_timeline()
try:
    public_tweets = api.mentions_timeline()
except tweepy.error.TweepError as e:
    time.sleep(180)
    public_tweets = api.mentions_timeline()
    
print(len(public_tweets))
#look at first tweet only for now
idx= 0 ;

#loop through twwets in timeline
for i in range(1):
    print("loop: " + str(i))
    for idx,tweet in enumerate(public_tweets):
       
        #print("found tweet :" + str(idx))
        print(tweet.text)
        #break on other tweets
        if idx >0:
            break;

        #split tweet to get username
        try:
            buffer_str =str(tweet.text).split()
        except tweepy.error.TweepError as e:
            time.sleep(180)
            buffer_str =str(tweet.text).split()

        #get into first tweet
        if idx == 0:
                #m = buffer_str[0] + " podcast"
                #print(m)
                #print(tweet.id)
                #print('tweet content : ' + str(tweet.text))

                
                #user info
                sn = str()
                try:
                    #user = api.get_user(tweet.id)
                    #user screen name
                    sn = tweet.user.screen_name
                except tweepy.error.TweepError as e:
                    time.sleep(180)
                    sn = tweet.user.screen_name
                    #user screen name
               
                
                
                print(sn)
                #print("sending recommendation to @" + str(sn))

                #get all twitter history from user
                #all_tweets = get_user_tweets(sn)
                #print("getting obama tweets")
             #   try:
              #      all_tweets = get_user_tweets('BarackObama')
               # except tweepy.error.TweepError as e:
                #    time.sleep(180)
                 #   all_tweets = get_user_tweets('BarackObama')
                    
                m = "@%s check this out  %s !" % (sn,podcast_url)
                #m = u"haha " + u'https://itunes.apple.com/us/podcast/ted-talks-daily/id160904630?mt=2&uo=4'
                #print("done getting obama tweets")
                #ok now notify user , case 1  : user has no much history

                if sn != "PFather101" and len(all_tweets) < 10:
                    try:
                        s = api.update_status(m)
                    except tweepy.error.TweepError as e:
                        print("going to sleep 1")
                        time.sleep(60*15)
                        s = api.update_status(m )
                    
                    print("reply sent going to sleep")
                    #time.sleep(1*60)
                    print("awake")

                #case 2 : user has history
                if sn != "PFather101" and len(all_tweets) > 10:
                    try:
                       # s = api.update_status(m, tweet.id)
                         s = api.update_status(m,tweet.id)
                    except tweepy.error.TweepError as e:
                        print("going to sleep 2")
                        time.sleep(60*15)
                        #s = api.update_status(m, tweet.id) 
                        s = api.update_status(m, tweet.id)
                        
                    print("reply sent to @" + str(sn))
                    print("reply sent going to sleep")
                    #time.sleep(1*60)
                    print("awake")
        idx= idx+1
            


8
loop: 0
@PFather101 hey you you
jandresmillan
going to sleep 1


KeyboardInterrupt: 

In [4]:
public_tweets = api.home_timeline()


In [5]:
public_tweets = api.home_timeline()
idx= 0 ;
for tweet in public_tweets:
    print(str(tweet.text))
    buffer_str =str(tweet.text).split()
    if idx == 0:
            m = buffer_str[0] + " podcast"
            print(tweet.id)
            user = api.get_user(buffer_str[0])
            sn = user.screen_name
            m = "@%s check this out %s !" % (sn,podcast_url)
            s = api.update_status(m, tweet.id)
    idx= idx+1

In [7]:
public_tweets

[]